# Arquivo de coleta de dados do BenchMark

## Atualizando o path do sistema

In [ ]:
import os
import sys

current_dir: str = os.path.abspath(os.curdir)
if current_dir not in sys.path:
    sys.path.insert(0, current_dir)
